In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
import string
import re
#import emoji #если понадобятся

In [2]:
dtype = {'id': np.uint16, 'date': str, 'name': str, 'text': str,  'type': np.uint8}

ppath = 'C:/Users/user/sentiment/positive.csv' #home
npath = 'C:/Users/user/sentiment/negative.csv' #home

In [3]:
pdf = pd.read_csv (ppath, sep=';', names=['id', 'date', 'name', 'text', 'type'], usecols=dtype.keys(), dtype=dtype)
ndf = pd.read_csv (npath, sep=';', names=['id', 'date', 'name', 'text', 'type'], usecols=dtype.keys(), dtype=dtype)

In [4]:
pdf.drop(['id', 'date', 'name'], axis=1, inplace=True)
ndf.drop(['id', 'date', 'name'], axis=1, inplace=True)

In [5]:
pdf.head()

,text,type
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [6]:
import re
import unittest


class Stemmer: #for Russian 1/2
    # Helper regex strings.
    _vowel = "[аеиоуыэюя]"
    _non_vowel = "[^аеиоуыэюя]"

    # Word regions.
    _re_rv = re.compile(_vowel)
    _re_r1 = re.compile(_vowel + _non_vowel)

    # Endings.
    _re_perfective_gerund = re.compile(
        r"(((?P<ignore>[ая])(в|вши|вшись))|(ив|ивши|ившись|ыв|ывши|ывшись))$"
    )
    _re_adjective = re.compile(
        r"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|"
        r"ую|юю|ая|яя|ою|ею)$"
    )
    _re_participle = re.compile(
        r"(((?P<ignore>[ая])(ем|нн|вш|ющ|щ))|(ивш|ывш|ующ))$"
    )
    _re_reflexive = re.compile(
        r"(ся|сь)$"
    )
    _re_verb = re.compile(
        r"(((?P<ignore>[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|"
        r"нно))|(ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|"
        r"ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю))$"
    )
    _re_noun = re.compile(
        r"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|"
        r"ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$"
    )
    _re_superlative = re.compile(
        r"(ейш|ейше)$"
    )
    _re_derivational = re.compile(
        r"(ост|ость)$"
    )
    _re_i = re.compile(
        r"и$"
    )
    _re_nn = re.compile(
        r"((?<=н)н)$"
    )
    _re_ = re.compile(
        r"ь$"
    )

    def stem(self, word):
        """
        Gets the stem.
        """

        rv_pos, r2_pos = self._find_rv(word), self._find_r2(word)
        word = self._step_1(word, rv_pos)
        word = self._step_2(word, rv_pos)
        word = self._step_3(word, r2_pos)
        word = self._step_4(word, rv_pos)
        return word

    def _find_rv(self, word):
        """
        Searches for the RV region.
        """

        rv_match = self._re_rv.search(word)
        if not rv_match:
            return len(word)
        return rv_match.end()

    def _find_r2(self, word):
        """
        Searches for the R2 region.
        """

        r1_match = self._re_r1.search(word)
        if not r1_match:
            return len(word)
        r2_match = self._re_r1.search(word, r1_match.end())
        if not r2_match:
            return len(word)
        return r2_match.end()

    def _cut(self, word, ending, pos):
        """
        Tries to cut the specified ending after the specified position.
        """

        match = ending.search(word, pos)
        if match:
            try:
                ignore = match.group("ignore") or ""
            except IndexError:
                # No ignored characters in pattern.
                return True, word[:match.start()]
            else:
                # Do not cut ignored part.
                return True, word[:match.start() + len(ignore)]
        else:
            return False, word

    def _step_1(self, word, rv_pos):
        match, word = self._cut(word, self._re_perfective_gerund, rv_pos)
        if match:
            return word
        _, word = self._cut(word, self._re_reflexive, rv_pos)
        match, word = self._cut(word, self._re_adjective, rv_pos)
        if match:
            _, word = self._cut(word, self._re_participle, rv_pos)
            return word
        match, word = self._cut(word, self._re_verb, rv_pos)
        if match:
            return word
        _, word = self._cut(word, self._re_noun, rv_pos)
        return word

    def _step_2(self, word, rv_pos):
        _, word = self._cut(word, self._re_i, rv_pos)
        return word

    def _step_3(self, word, r2_pos):
        _, word = self._cut(word, self._re_derivational, r2_pos)
        return word

    def _step_4(self, word, rv_pos):
        _, word = self._cut(word, self._re_superlative, rv_pos)
        match, word = self._cut(word, self._re_nn, rv_pos)
        if not match:
            _, word = self._cut(word, self._re_, rv_pos)
        return word

In [7]:
from nltk.corpus import stopwords
#from nltk.stem import PorterStemmer # for English 1/2

stops = set(stopwords.words("russian"))

def clean_text(data):
    low = True 
    delstops = True
    stem = True
    del12gram = True  # True = убираем обрывки слов в 1-2 символа
    
    data = re.sub(r'ё', r'е', data) # ё ----> е
    data = re.sub(r'Ё', r'Е', data)
    data = re.sub(r'[^а-яА-Я ]',r' ',data) #оставляем только русские буквы и пробелы (все ост символы заменяются на пробел)
    data = ' '.join(data.split()) #убираем лишние пробелы
    
    if low:
        data = data.lower()
    
    if delstops:
        data = " ".join([w for w in data.split() if w not in stops])
    
    if stem:
        #st = PorterStemmer() #for English 2/2
        st = Stemmer() #for Russian 2/2
        data = " ".join([st.stem(w) for w in data.split()])
    
    if del12gram:
        data = " ".join([w for w in data.split() if len(w) > 2])
        
    return data

In [8]:
pdf['clear_text'] = pdf['text'].map(clean_text)
ndf['clear_text'] = ndf['text'].map(clean_text)

In [9]:
pdf.drop('text', axis=1, inplace=True)
ndf.drop('text', axis=1, inplace=True)

In [10]:
ndf['type'] = -1

In [11]:
df = pd.concat([pdf, ndf], ignore_index=True)
df

,type,clear_text
0,1,школот повер сам обществ профилир предмет тип
1,1,так немн похож мальчик равн
2,1,идиотк испуга
3,1,угл сид погиба голод порц взял хот жрат хот
4,1,знач страшилк блин посмотр част создаст ощущен...
5,1,люб зна бля
6,1,друз так мим
7,1,ухажер жрет
8,1,поприветству нов читател
9,1,частичк сидне


In [125]:
df.to_csv('clear_binary_twitts.csv', index=False)